In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import datetime as dt
import string

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context("poster")
sns.set_style("ticks")
sns.set(font_scale=2)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 15000)
pd.set_option('display.max_colwidth', -1)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

### RFID data

#### Get paths for files to process

In [ ]:
#create list of dir paths
RFID_path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID_VDM/AQP4/RFID'
    
RFID_file_names = os.listdir(RFID_path)

RFID_data_dir_paths = []

for file in RFID_file_names:
    int_path = RFID_path + '/' + file
    RFID_data_dir_paths.append(int_path)
    
RFID_data_dir_paths

#### Clean df and fix datetime

In [ ]:
path = RFID_data_dir_paths[1]

data_RFID_int = pd.read_csv(path, sep=';', encoding='utf-16')

data_RFID_int = pd.DataFrame(data = data_RFID_int)
print(data_RFID_int.shape)

data_RFID_int.head(50)

In [ ]:
row_drop = 33

data_RFID = pd.DataFrame()

for path in RFID_data_dir_paths:
    print(path)

    if path.split('/')[-1] == '.DS_Store':
        continue
    
    #data_RFID_int['file'] = path.split('/')[-1]
    
    #create data table
    data_RFID_int = pd.read_csv(path, sep=';', encoding='utf-16')
    data_RFID_int = pd.DataFrame(data = data_RFID_int)
    data_RFID_int = data_RFID_int.iloc[row_drop: , :]
    print(data_RFID_int.shape)
    
    #add file name
    data_RFID_int['RFID_file'] = path.split('/')[-1]
                                            
    #drop rows with no event duration
    data_RFID_int = data_RFID_int[~data_RFID_int['eventDuration'].isna()]
    print(data_RFID_int.shape)
    data_RFID_int = data_RFID_int[~data_RFID_int['DateTime'].isna()]
    print(data_RFID_int.shape)

    #convert from ms to sec
    data_RFID_int['eventDuration'] = data_RFID_int['eventDuration']/1000
    #clean up and create datetime columns
    data_RFID_int['start_datetime'] = pd.to_datetime(data_RFID_int['DateTime'], unit='d')
    #can't figure out datetime format so hard code for now
    data_RFID_int['RFID_datetime_start'] = [x.replace(year = 2023) - dt.timedelta(days = 1) for x in data_RFID_int['start_datetime']] 
    data_RFID_int['RFID_date'] = data_RFID_int['RFID_datetime_start'].dt.date 
    data_RFID_int['RFID_day'] = data_RFID_int['RFID_datetime_start'].dt.day 
    data_RFID_int['RFID_time_start'] = [x.split(' ')[1] for x in data_RFID_int['RFID_datetime_start'].dt.strftime("%Y-%m-%d %H:%M:%S")]
    data_RFID_int['RFID_hour'] = data_RFID_int['RFID_datetime_start'].dt.hour
    data_RFID_int['RFID_min'] = data_RFID_int['RFID_datetime_start'].dt.minute
    data_RFID_int['RFID_second'] = data_RFID_int['RFID_datetime_start'].dt.second

    if data_RFID.empty:
        data_RFID = data_RFID_int
    else:
        data_RFID = pd.concat([data_RFID, data_RFID_int], axis=0)
        
    print(data_RFID.shape,'\n')

#clean columns
data_RFID = data_RFID[['RFID_file', 'IdRFID', 'unitLabel', 'eventDuration',
       'RFID_datetime_start', 'RFID_date',
       'RFID_day', 'RFID_time_start', 'RFID_hour', 'RFID_min', 'RFID_second']]

print(data_RFID.shape,'\n')

data_RFID.head()

In [ ]:
data_RFID['RFID_day'].value_counts()

In [ ]:
data_RFID['IdRFID'].value_counts()

#### Compute endtime 

In [ ]:
#compute end time for each eventDuration
def add_duration(a, b):
    newtime = a + dt.timedelta(0, b)

    return newtime

data_RFID['RFID_datetime_end'] = data_RFID.apply(lambda row: add_duration(row['RFID_datetime_start'], row['eventDuration']), axis=1)
data_RFID['RFID_time_end'] = [x.split(' ')[1] for x in data_RFID['RFID_datetime_end'].dt.strftime("%Y-%m-%d %H:%M:%S")]

print(data_RFID.shape)
data_RFID.head()

#### Add day, hour, and min counts

In [ ]:
#column of running day count
dat_map = dict(zip(data_RFID['RFID_date'].sort_values().unique(), np.arange(0,len(data_RFID['RFID_date'].unique()))))
data_RFID['RFID_day_count'] = data_RFID['RFID_date'].map(dat_map)

#column of running min and second count within each day
data_RFID['RFID_min_count'] = (data_RFID['RFID_hour']*60) + data_RFID['RFID_min'] 
data_RFID['RFID_sec_count'] = (data_RFID['RFID_min_count']*60) + data_RFID['RFID_second'] 

#column of running hour and minute count across all days
data_RFID['RFID_hour_count_running'] = (data_RFID['RFID_day_count']*24) + data_RFID['RFID_hour'] - 24
data_RFID['RFID_min_count_running'] = (data_RFID['RFID_hour_count_running']*60) + data_RFID['RFID_min'] 
data_RFID['RFID_sec_count_running'] = (data_RFID['RFID_min_count_running']*60) + data_RFID['RFID_second'] - 60

print(data_RFID.shape)
data_RFID.head()

In [ ]:
dat_map

#### Light/dark mapping

In [ ]:
def assign_ld(time):
    if (time >= 6) & (time < 18):
        return 'light'
    if (time >= 0) & (time < 6):
        return 'dark_am'
    if (time >= 18):
        return 'dark_pm'
    
data_RFID['ld_seg'] = data_RFID['RFID_hour'].apply(assign_ld)

print(data_RFID.shape)
data_RFID.head()


In [ ]:
ld_mapping_path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID_VDM/AQP4/meta/ld_mapping.xlsx'

ld_mapping = pd.read_excel(ld_mapping_path)
ld_mapping = pd.DataFrame(data = ld_mapping)
print(ld_mapping.shape)
ld_mapping.head()

In [ ]:
data_RFID = data_RFID.merge(ld_mapping, right_on=['day', 'ld_seg'], left_on=['RFID_day_count', 'ld_seg'])
print(data_RFID.shape)
data_RFID.head()

In [ ]:
sns.displot(data=data_RFID, x="eventDuration", kind="ecdf", hue='IdRFID', col='ld_cycle')
plt.xlabel('Visit Duration (sec)')
plt.xlim(0, 60)
plt.show()

In [ ]:
data_RFID.groupby(['unitLabel'])['IdRFID'].value_counts()

#### Map unitLabel and bottle contents

In [ ]:
RFID_bottle_assignment_path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID_VDM/AQP4/meta/bottle_assignment_RFID.xlsx'

RFID_bottle_assignment = pd.read_excel(RFID_bottle_assignment_path)
RFID_bottle_assignment = pd.DataFrame(data = RFID_bottle_assignment)
print(RFID_bottle_assignment.shape)
RFID_bottle_assignment.tail()

In [ ]:
data_RFID_bottle = pd.DataFrame()

for file in data_RFID['RFID_file'].unique():
    print(file)
    d_int = data_RFID[data_RFID['RFID_file']==file]
    
    RFID_bottle_map = RFID_bottle_assignment[RFID_bottle_assignment['RFID_file']==file].reset_index().loc[0,'Spyder1.01':'Spyder1.32'].to_dict()
    d_int['unitLabel_drink'] = d_int['unitLabel'].map(RFID_bottle_map)
    
    if data_RFID_bottle.empty:
        data_RFID_bottle = d_int
    else:
        data_RFID_bottle = pd.concat([data_RFID_bottle, d_int], axis=0)
        
    print(data_RFID_bottle.shape,'\n')
    
data_RFID_bottle.head()

In [ ]:
data_RFID = data_RFID_bottle

data_RFID.groupby('unitLabel_drink')['unitLabel'].value_counts()

#### Add metadata and clean

In [ ]:
meta_path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID_VDM/AQP4/meta/meta_data.xlsx'
meta_data = pd.read_excel(meta_path)
meta_data = pd.DataFrame(data = meta_data)
meta_data['IdRFID'] = [str(x) for x in meta_data['IdRFID']]
data_RFID_final = meta_data.merge(data_RFID.reset_index(), on='IdRFID', sort=False)

print(data_RFID_final.shape)
data_RFID_final.head()

In [ ]:
data_RFID_final.columns.values

In [ ]:
data_RFID_final = data_RFID_final[['IdRFID', 'Genotype', 'Sex', 'ld_cycle', 'ld_day',
                                   'unitLabel', 'eventDuration', 'unitLabel_drink',
                                   'RFID_datetime_start', 'RFID_time_start', 'RFID_datetime_end', 'RFID_time_end',
                                   'RFID_date', 'RFID_day', 'RFID_hour', 'RFID_min', 'RFID_second', 
                                   'RFID_day_count', 'RFID_min_count', 'RFID_sec_count',
                                   'RFID_hour_count_running', 'RFID_min_count_running', 'RFID_sec_count_running',
                                   'Cage', 'Animal', 'ARF_Cage', 'Round', 'Startdate', 'Cage_N', 'RFID_file', 
                                   'wt1', 'wt2', 'wt_change']]

print(data_RFID_final.shape)
data_RFID_final.head()

In [ ]:
data_RFID_final.groupby('Sex')['Cage'].value_counts()

In [ ]:
data_RFID_final.to_csv('RFID_231211.csv')

#### Viz

In [ ]:
data_int = data_RFID_final
sns.displot(data=data_int, x="eventDuration", kind="ecdf", hue='Genotype')
plt.xlabel('Visit Duration (sec)')
plt.xlim(0, 60)
plt.show()

In [ ]:
data_int = data_RFID_final

data_int = data_int.groupby(['Sex', 'IdRFID', 'Genotype', 'ld_day', 'unitLabel_drink'])['eventDuration'].sum().reset_index(name='count')

plt.figure(figsize=(10,5))
params = {'axes.labelsize': 30,
          'axes.titlesize': 30}
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 
sns.catplot(data=data_int, x="unitLabel_drink", y='count', hue="Genotype", kind='bar', height=5, aspect=2, 
            ci=68, col='Sex')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3), 
          ncol=5, fancybox=True, shadow=True)
plt.xlabel('Substance type')
plt.ylabel('Visit count')
plt.show()

### VDM data

#### Get paths for files to process

In [ ]:
#create list of dir paths
VDM_path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID_VDM/AQP4/VDM'
    
VDM_file_names = os.listdir(VDM_path)

VDM_data_dir_paths = []

for file in VDM_file_names:
    int_path = VDM_path + '/' + file
    VDM_data_dir_paths.append(int_path)
    
VDM_data_dir_paths

#### Clean df and remove starting meta data

In [ ]:
data_VDM_int = pd.read_table(VDM_data_dir_paths[0], sep=';')
data_VDM_int = pd.DataFrame(data = data_VDM_int)
data_VDM_int.head(50)

In [ ]:
row_drop = 22
data_VDM = pd.DataFrame()

for path in VDM_data_dir_paths:
    print(path)
    
    if path.split('/')[-1] == '.DS_Store':
        continue
        
    #create data table
    data_VDM_int = pd.read_table(path, sep=';')
    data_VDM_int = pd.DataFrame(data = data_VDM_int)
    data_VDM_int = data_VDM_int.iloc[row_drop: , :]['Columbus Instruments Device Interface Version 1.5'].str.split(',', expand=True)
    data_VDM_int.columns = data_VDM_int.loc[row_drop].values
    data_VDM_int = data_VDM_int.drop(row_drop)
    
    data_VDM_int['VDM_file'] = path.split('/')[-1]

    if data_VDM.empty:
        data_VDM = data_VDM_int
    else:
        data_VDM = pd.concat([data_VDM, data_VDM_int], axis=0)
        
    print(data_VDM.shape,'\n')

print(data_VDM.shape)
data_VDM.head()

In [ ]:
data_VDM.columns = ['Int', 'Cage_VDM', 'VDM_Time', 
                    'drink_1', 'drink_1_accum',
                    'drink_2', 'drink_2_accum',
                    'drink_3', 'drink_3_accum',
                    'drink_4', 'drink_4_accum',
                    'blank', 'VDM_file']

data_VDM = data_VDM[['VDM_file', 'Cage_VDM', 'VDM_Time', 
                    'drink_1', 
                    'drink_2', 
                    'drink_3', 
                    'drink_4', 
                    ]]

data_VDM.tail()

#### Make into tidy df

In [ ]:
#melt dataframe (new column of parameters)

meta_params = ['VDM_file', 'Cage_VDM', 'VDM_Time']
intake_params = ['drink_1', 'drink_2', 'drink_3', 'drink_4']

data_VDM_tidy = pd.melt(data_VDM, id_vars=meta_params, value_vars=intake_params)

data_VDM_tidy = data_VDM_tidy.replace({'':0.000})

data_VDM_tidy['value'] = data_VDM_tidy['value'].astype('float')

print(data_VDM_tidy.shape)
data_VDM_tidy.head()

In [ ]:
data_VDM_tidy['value'].unique()

#### Drop times with no intake data

In [ ]:
data_VDM_tidy_intake = data_VDM_tidy[data_VDM_tidy['value'] > 0.000]
print(data_VDM_tidy_intake.shape)
data_VDM_tidy_intake.head()

#### Clean up datetime

In [ ]:
#clean up and create datetime columns
data_VDM_tidy_intake['VDM_datetime'] = pd.to_datetime(data_VDM_tidy_intake['VDM_Time'])
data_VDM_tidy_intake['VDM_timestamp'] = [x.timestamp() for x in data_VDM_tidy_intake['VDM_datetime']]

data_VDM_tidy_intake['VDM_date'] = data_VDM_tidy_intake['VDM_datetime'].dt.date
data_VDM_tidy_intake['VDM_day'] = data_VDM_tidy_intake['VDM_datetime'].dt.day
data_VDM_tidy_intake['VDM_time'] = data_VDM_tidy_intake['VDM_datetime'].dt.strftime("%H:%M:%S")
data_VDM_tidy_intake['VDM_hour'] = data_VDM_tidy_intake['VDM_datetime'].dt.hour
data_VDM_tidy_intake['VDM_min'] = data_VDM_tidy_intake['VDM_datetime'].dt.minute
data_VDM_tidy_intake['VDM_second'] = data_VDM_tidy_intake['VDM_datetime'].dt.second

print(data_VDM_tidy_intake.shape)
data_VDM_tidy_intake.head()

#### Add day, hour, and min counts

In [ ]:
#column of running day count
dat_map = dict(zip(data_VDM_tidy_intake['VDM_date'].sort_values().unique(), np.arange(0,len(data_VDM_tidy_intake['VDM_date'].unique()))))
data_VDM_tidy_intake['VDM_day_count'] = data_VDM_tidy_intake['VDM_date'].map(dat_map)

#column of running min and second count within each day
data_VDM_tidy_intake['VDM_min_count'] = (data_VDM_tidy_intake['VDM_hour']*60) + data_VDM_tidy_intake['VDM_min'] 
data_VDM_tidy_intake['VDM_sec_count'] = (data_VDM_tidy_intake['VDM_min_count']*60) + data_VDM_tidy_intake['VDM_second'] 

#column of running hour and minute count across all days
data_VDM_tidy_intake['VDM_hour_count_running'] = (data_VDM_tidy_intake['VDM_day_count']*24) + data_VDM_tidy_intake['VDM_hour'] - 24
data_VDM_tidy_intake['VDM_min_count_running'] = (data_VDM_tidy_intake['VDM_hour_count_running']*60) + data_VDM_tidy_intake['VDM_min'] 
data_VDM_tidy_intake['VDM_sec_count_running'] = (data_VDM_tidy_intake['VDM_min_count_running']*60) + data_VDM_tidy_intake['VDM_second'] - 60

print(data_VDM_tidy_intake.shape)
data_VDM_tidy_intake.head()

In [ ]:
data_VDM_tidy_intake['VDM_day'].value_counts()

#### Light/dark mapping

In [ ]:
def assign_ld(time):
    if (time >= 6) & (time < 18):
        return 'light'
    if (time >= 0) & (time < 6):
        return 'dark_am'
    if (time >= 18):
        return 'dark_pm'
    
data_VDM_tidy_intake['ld_seg'] = data_VDM_tidy_intake['VDM_hour'].apply(assign_ld)

data_VDM_tidy_intake.head()

In [ ]:
data_VDM_tidy_intake = data_VDM_tidy_intake.merge(ld_mapping, right_on=['day', 'ld_seg'], left_on=['VDM_day_count', 'ld_seg'])
print(data_VDM_tidy_intake.shape)
data_VDM_tidy_intake.head()

In [ ]:
dat_map

#### Map bottle contents

In [ ]:
data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='1') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_H20_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='1') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_EtOH10'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='1') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_Fent20'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='1') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_H20_2'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='2') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_EtOH05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='2') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_Fent05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='2') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_drip_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='2') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_drip_2'


data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='3') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_H20_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='3') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_EtOH10'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='3') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_Fent20'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='3') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_H20_2'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='4') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_EtOH05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='4') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_Fent05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='4') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_drip_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='4') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_drip_2'



data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='5') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_H20_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='5') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_EtOH10'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='5') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_Fent20'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='5') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_H20_2'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='6') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_EtOH05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='6') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_Fent05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='6') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_drip_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='6') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_drip_2'


data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='7') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_H20_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='7') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_EtOH10'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='7') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_Fent20'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='7') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_H20_2'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='8') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_EtOH05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='8') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_Fent05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='8') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_drip_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='8') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_drip_2'


print(data_VDM_tidy_intake.shape)
data_VDM_tidy_intake.head()

In [ ]:

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='1') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_Fent20'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='1') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_H20_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='1') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_EtOH10'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='1') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_Fent05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='2') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_H20_2'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='2') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_EtOH05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='2') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_drip_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='2') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_drip_2'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='3') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_Fent20'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='3') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_H20_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='3') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_EtOH10'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='3') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_Fent05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='4') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_H20_2'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='4') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_EtOH05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='4') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_drip_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='4') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_drip_2'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='5') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_Fent20'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='5') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_H20_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='5') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_EtOH10'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='5') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_Fent05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='6') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_H20_2'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='6') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_EtOH05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='6') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_drip_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='6') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_drip_2'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='7') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_Fent20'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='7') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_H20_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='7') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_EtOH10'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='7') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_Fent05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='8') & (data_VDM_tidy_intake['variable']=='drink_1'), 
                         'VDM_drink'] = 'VDM_H20_2'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='8') & (data_VDM_tidy_intake['variable']=='drink_2'), 
                         'VDM_drink'] = 'VDM_EtOH05'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='8') & (data_VDM_tidy_intake['variable']=='drink_3'), 
                         'VDM_drink'] = 'VDM_drip_1'

data_VDM_tidy_intake.loc[(data_VDM_tidy_intake['Cage_VDM']=='8') & (data_VDM_tidy_intake['variable']=='drink_4'), 
                         'VDM_drink'] = 'VDM_drip_2'

print(data_VDM_tidy_intake.shape)
data_VDM_tidy_intake.head()

In [ ]:
def assign_cage(cage):
    if cage == '1':
        return 1
    if cage == '2':
        return 1
    if cage == '3':
        return 2
    if cage == '4':
        return 2
    if cage == '5':
        return 3
    if cage == '6':
        return 3
    if cage == '7':
        return 4
    if cage == '8':
        return 4
    
data_VDM_tidy_intake['Cage_VDM_update'] = data_VDM_tidy_intake['Cage_VDM'].apply(assign_cage)

data_VDM_tidy_intake.head()

#### Viz

In [ ]:
data_int = data_VDM_tidy_intake.groupby(['Cage_VDM_update', 'VDM_drink'])['value'].sum().reset_index(name='sum')

plt.figure(figsize=(10,5))
params = {'axes.labelsize': 30,
          'axes.titlesize': 30}
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 
sns.catplot(data=data_int, x="VDM_drink", y='sum', kind='bar', height=5, aspect=2, 
            col='Cage_VDM_update', col_wrap=2, sharey=False, sharex=False)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3),
          ncol=5, fancybox=True, shadow=True)
plt.xlabel('Substance type')
plt.ylabel('Visit count')
plt.show()

### Filter RFID data by VDM times

In [ ]:
data_VDM_tidy_intake = data_VDM_tidy_intake.sort_values(['VDM_drink', 'VDM_datetime']).reset_index(drop=True)
drink_drop = ['VDM_drip_1', 'VDM_drip_2']
data_VDM_tidy_intake = data_VDM_tidy_intake[~data_VDM_tidy_intake['VDM_drink'].isin(drink_drop)]
print(data_VDM_tidy_intake.groupby('Cage_VDM_update')['VDM_drink'].value_counts())
print(data_VDM_tidy_intake.shape)
data_VDM_tidy_intake.head()

In [ ]:
data_RFID_final= data_RFID_final.sort_values(['unitLabel_drink', 'RFID_datetime_start']).reset_index(drop=True)
print(data_RFID_final['unitLabel_drink'].value_counts())
data_RFID_final.head()

#### Match cage, unitLabel, and datetime

In [ ]:
data_comb = data_VDM_tidy_intake

flavor_mapping = {'VDM_Fent05': 'fent_05',
                  'VDM_Fent20': 'fent_20',
                  'VDM_EtOH05': 'etoh_05',
                  'VDM_EtOH10': 'etoh_10',
                  'VDM_H20_1': 'water_1',
                  'VDM_H20_2': 'water_2'}

RFID_eventDuration = []
RFID_assign_id_pre = []
RFID_assign_var_pre = []
RFID_datetime_start_pre = []
RFID_datetime_end_pre = []

for index, row in data_VDM_tidy_intake.iterrows():
    if index%1000 == 0:
        print(index)
        
    #find RFID signal start time closest to VDM time before
    try:
        int_assign = data_RFID_final[(data_RFID_final['Cage'] == row['Cage_VDM_update']) & #match cage
                               (data_RFID_final['unitLabel_drink'] == flavor_mapping[row['VDM_drink']]) & #match tube
                               (data_RFID_final['RFID_datetime_start'] <= row['VDM_datetime']) #find closest datetime
                              ].sort_values(['RFID_datetime_start']).iloc[-1]

        RFID_eventDuration.append(int_assign['eventDuration'])
        RFID_assign_id_pre.append(int_assign['IdRFID'])
        RFID_assign_var_pre.append(int_assign['unitLabel_drink'])
        RFID_datetime_start_pre.append(int_assign['RFID_datetime_start'])
        RFID_datetime_end_pre.append(int_assign['RFID_datetime_end'])

        
    except:
        print(index, 'pre', row['Cage_VDM_update'])
        RFID_eventDuration.append(np.nan)
        RFID_assign_id_pre.append(np.nan)
        RFID_assign_var_pre.append(np.nan)
        RFID_datetime_start_pre.append(np.nan)
        RFID_datetime_end_pre.append(np.nan)

        
data_comb['RFID_eventDuration'] = RFID_eventDuration
data_comb['RFID_assign_id_pre'] = RFID_assign_id_pre
data_comb['RFID_assign_var_pre'] = RFID_assign_var_pre
data_comb['RFID_datetime_start_pre'] = RFID_datetime_start_pre
data_comb['RFID_datetime_end_pre'] = RFID_datetime_end_pre

print(data_comb.shape)
data_comb.head()

In [ ]:
data_comb.isna().sum()

In [ ]:
data_comb.groupby(['ld_day', 'VDM_drink'])['value'].sum()

In [ ]:
data_comb.groupby(['ld_day', 'ld_cycle'])['VDM_date'].value_counts()

#### Copmuter RFID/VDM time difference

In [ ]:
def time_within(RFID_start, RFID_stop, VDM_time):
    
    if (VDM_time >= RFID_start) & (VDM_time <= RFID_stop):
        return 0
    
    elif (VDM_time >= RFID_start) & (VDM_time >= RFID_stop):
        time_within = VDM_time - RFID_stop
        time_within = time_within.total_seconds()
        return time_within
    
data_comb['VDM_RFID_timediff'] = data_comb.apply(lambda row: time_within(row['RFID_datetime_start_pre'], 
                                                                   row['RFID_datetime_end_pre'],
                                                                   row['VDM_datetime']), axis=1)

data_comb['VDM_RFID_timediff_bool'] = ['same' if x==0 else 'diff' for x in data_comb['VDM_RFID_timediff']]

data_comb.head()

In [ ]:
data_comb.groupby(['Cage_VDM_update', 'VDM_drink', ])['VDM_RFID_timediff_bool'].value_counts()

In [ ]:
sns.displot(data=data_comb, x="VDM_RFID_timediff", kind="ecdf", hue='VDM_drink', col='Cage_VDM_update')

plt.xlim(-1, 60)
plt.show()

#### Add metadata

In [ ]:
data_comb = meta_data.merge(data_comb.reset_index(), left_on='IdRFID', right_on='RFID_assign_id_pre', sort=False)

print(data_comb.shape)
data_comb.head()

In [ ]:
data_comb['VDM_drink'] = data_comb['VDM_drink'].replace({'VDM_H20_1':'VDM_water'})
data_comb['VDM_drink'] = data_comb['VDM_drink'].replace({'VDM_H20_2':'VDM_water'})

In [ ]:
data_comb['substance'] = [x.split('_')[-1] for x in data_comb['VDM_drink']]

In [ ]:
data_comb.to_csv('poly_221010.csv')

In [ ]:
data_int = data_VDM_tidy_intake[data_VDM_tidy_intake['VDM_RFID_timediff']<3].groupby(['Cage_VDM_update', 'VDM_hour'])['value'].sum().reset_index(name='sum')

plt.figure(figsize=(10,5))
params = {'axes.labelsize': 30,
          'axes.titlesize': 30}
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 
sns.catplot(data=data_int, x="VDM_hour", y='sum', kind='bar', height=5, aspect=2, 
            col='Cage_VDM_update', col_wrap=2, sharey=False, sharex=False)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3),
          ncol=5, fancybox=True, shadow=True)
plt.xlabel('Substance type')
plt.ylabel('Visit count')
plt.show()